In [1]:
#import data
import os
import glob
import boto3
from pathlib import Path
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.xgboost import XGBoost, XGBoostPredictor


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
def download_imdb_data():
    """Download IMDb dataset if not already present"""
    data_dir = Path('../data/aclImdb')
    if not data_dir.exists():
        print("Downloading IMDb dataset...")
        # In production, we'd use a more reliable source
        os.system('wget -P ../data/ https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
        os.system('tar -xvzf ../data/aclImdb_v1.tar.gz -C ../data/')
    return str(data_dir)




In [3]:
def create_processing_step(role):
    """Create SageMaker Processing Job"""
    sklearn_processor = SKLearnProcessor(
        framework_version='1.2-1',
        role=role,
        instance_type='ml.m5.large',
        instance_count=1,
        base_job_name='imdb-processing'
    )
    
    return ProcessingStep(
        name='IMDBDataProcessing',
        processor=sklearn_processor,
        inputs=[
            ProcessingInput(
                source=download_imdb_data(),
                destination='/opt/ml/processing/input/data'
            )
        ],
        outputs=[
            ProcessingOutput(output_name='train', source='/opt/ml/processing/output/train'),
            ProcessingOutput(output_name='validation', source='/opt/ml/processing/output/validation'),
            ProcessingOutput(output_name='test', source='/opt/ml/processing/output/test'),
            ProcessingOutput(output_name='vectorizer', source='/opt/ml/processing/output/vectorizer')
        ],
        code='scripts/processing_job.py'
    )

In [4]:
def create_training_step(train_s3, val_s3, config):
    """Create SageMaker Training Job"""
    container = sagemaker.image_uris.retrieve('xgboost', config["region"], '1.7-1')
    
    hyperparameters = {
        'max_depth': '5',
        'eta': '0.2',
        'gamma': '4',
        'min_child_weight': '6',
        'subsample': '0.8',
        'objective': 'binary:logistic',
        'early_stopping_rounds': '10',
        'num_round': '100'
    }
    
    xgb_estimator = XGBoost(
        entry_point='train.py',
        hyperparameters=hyperparameters,
        image_uri=container,
        role=config["role"],
        instance_count=1,
        instance_type='ml.m5.2xlarge',
        framework_version='1.7-1',
        output_path=f's3://{config["bucket"]}/{config["prefix"]}/models',
        use_spot_instances=True,
        max_wait=7200,
        max_run=3600
    )
    
    return TrainingStep(
        name='IMDBModelTraining',
        estimator=xgb_estimator,
        inputs={
            'train': TrainingInput(s3_data=train_s3, content_type='text/csv'),
            'validation': TrainingInput(s3_data=val_s3, content_type='text/csv')
        }
    )


In [5]:
def main():
    # Initialize SageMaker
    session = sagemaker.Session()
    role = get_execution_role()
    region = session.boto_region_name
    bucket = "sentiment-analysis-workflow"
    prefix = 'imdb-sentiment-analysis'
    
    config = {
    "session": session,
    "role": role,
    "region": region,
    "bucket": bucket,
    "prefix": prefix
}
    
    processing_step = create_processing_step(role)
    
    train_s3 = processing_step.properties.ProcessingOutputConfig.Outputs['train'].S3Output.S3Uri
    val_s3 = processing_step.properties.ProcessingOutputConfig.Outputs['validation'].S3Output.S3Uri
    training_step = create_training_step(train_s3, val_s3,  config)
    
    pipeline = Pipeline(
        name='IMDBSentimentPipeline',
        steps=[processing_step, training_step],
        sagemaker_session=session
    )
    
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()
    execution.wait()
    
    model_data = execution.steps[1].properties.ModelArtifacts.S3ModelArtifacts
    endpoint_name = 'imdb-sentiment-endpoint'
    
    # Deploy model
    model = sagemaker.model.Model(
        image_uri=sagemaker.image_uris.retrieve('xgboost', region, '1.7-1'),
        model_data=model_data,
        role=role,
        predictor_cls=XGBoostPredictor
    )
    
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.large',
        endpoint_name=endpoint_name
    )
    
    print("✅ End-to-end workflow completed!")

    
main()

# if __name__ == "__main__":
#     main()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:53                                                                                   │
│                                                                                                  │
│   50 │   print("✅ End-to-end workflow completed!")                                              │
│   51                                                                                             │
│   52                                                                                             │
│ ❱ 53 main()                                                                                      │
│   54                                                                                             │
│   55 # if __name__ == "__main__":                                                                │
│   56 #     main()                                                                                │
│                                                                                                  │
│ in main:29                                                                                       │
│                                                                                                  │
│   26 │   │   sagemaker_session=session                                                           │
│   27 │   )                                                                                       │
│   28 │                                                                                           │
│ ❱ 29 │   pipeline.upsert(role_arn=role)                                                          │
│   30 │   execution = pipeline.start()                                                            │
│   31 │   execution.wait()                                                                        │
│   32                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline.p │
│ y:292 in upsert                                                                                  │
│                                                                                                  │
│    289 │   │   │   # after fetching the config.                                                  │
│    290 │   │   │   raise ValueError("An AWS IAM role is required to create or update a Pipeline  │
│    291 │   │   try:                                                                              │
│ ❱  292 │   │   │   response = self.create(role_arn, description, tags, parallelism_config)       │
│    293 │   │   except ClientError as ce:                                                         │
│    294 │   │   │   error_code = ce.response["Error"]["Code"]                                     │
│    295 │   │   │   error_message = ce.response["Error"]["Message"]                               │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline.p │
│ y:164 in create                                                                                  │
│                                                                                                  │
│    161 │   │   tags = format_tags(tags)                                                          │
│    162 │   │   tags = _append_project_tags(tags)                                                 │
│    163 │   │   tags = self.sagemaker_session._append_sagemaker_config_tags(tags, PIPELINE_TAGS_  │
│ ❱  164 │   │   kwargs = self._create_args(role_arn, description, parallelism_config)             │
│    165 │   │   update_args(                                 